# NZ Admin jobs data exploration

## 1. Load data and overview

In [1]:
import pandas as pd
import re

In [2]:
# load file
df_admin = pd.read_excel('NZ_Admin_JOBS.xlsx')

In [3]:
# get info
df_admin.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   字段1       2708 non-null   object
 1   字段1_link  2708 non-null   object
 2   字段2       2686 non-null   object
 3   字段3       2708 non-null   object
 4   字段4       2708 non-null   object
 5   字段5       2708 non-null   object
dtypes: object(6)
memory usage: 127.1+ KB


In [4]:
# check missing
df_admin.isnull().sum().sort_values(ascending=False)

字段2         22
字段1          0
字段1_link     0
字段3          0
字段4          0
字段5          0
dtype: int64

In [5]:
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows',None)

#### Total 2,708 observations, 6 columns
#### No column name, all data are object

In [6]:
df_admin.describe()

,字段1,字段1_link,字段2,字段3,字段4,字段5
count,2708,2708,2686,2708,2708,2708
unique,548,2708,475,147,54,93
top,Executive Assistant,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Beyond Recruitment - Winner – Best Innovation –2018 Global Recruiter Awards,location: AucklandAucklandarea: Auckland CentralAuckland Central,"27d ago,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants
freq,301,1,285,654,572,715


#### '字段1_link' could be index
#### 22 Missing value in '字段2'
#### Column '字段1' contains position
#### Column '字段1_link'contains link
#### Column '字段2'  contains company name
#### Column '字段3' contains location
#### Column '字段4' shows release time
#### Column '字段5' contains position classification

In [7]:
df_admin.head(5)

,字段1,字段1_link,字段2,字段3,字段4,字段5
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: InvercargillInvercargill,"1h ago,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: CanterburyCanterburyarea: ChristchurchChristchurch,"4d ago,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration


### Poor data quality
#### 1. No column name;
#### 2. No index;
#### 3. Null value;
#### 4. Duplicate value;
#### 5. Upper case and lower case mixed;
#### 6. Abbreviation and full name mixed;
#### 7. Time data is not clear
#### 8. Missing data
#### 9. Data mixed in one column
#### 10. Salary format is different;
#### 11. Classification is too long. Parent classification are the same.

## 2. Data preprocess

#### 2.1 Add column name

In [8]:
# add column name first
column_name = ['Position','Link','Company','Location','Release_time','Classification']
df_admin.columns = column_name
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management


#### 2.2 Clean column 'Position'

#### 'office administrator' and 'Office Administrator' should be the same position

#### 2.2.1 Capitalize each word in Column 'Position'

In [9]:
# Capitalize the value
df_admin['Position'] = df_admin['Position'].str.title()
df_admin.head(5)

,Position,Link,Company,Location,Release_time,Classification
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,location: AucklandAuckland,"4d ago,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,location: SouthlandSouthlandarea: InvercargillInvercargill,"1h ago,at",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: CanterburyCanterburyarea: ChristchurchChristchurch,"4d ago,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration


#### Column 'Position' inlcude work type description, like 'full time','part time','10hours per week','temp'...
#### 2.2.2 Create and fill in Column 'Work type'

In [10]:
# select work type
print(df_admin['Position'].str.contains("[Ff]ull").sum()) #5
print(df_admin['Position'].str.contains("[Pp]art").sum()) #41
print(df_admin['Position'].str.contains("[Ff]ix").sum()) #20
print(df_admin['Position'].str.contains("[Pp]er").sum()) #308
print(df_admin['Position'].str.contains("[Cc]ontract").sum()) #96
df_admin['Worktype'] = df_admin.loc[df_admin['Position'].str.contains("[Ff]ull|[Pp]art|[Ff]ix|[Pp]er|[Cc]ontract"),'Position']


5
41
20
308
96


#### 2.2.3 Clean column 'Link'
#### job/50582301? number between'/' and '?' seems company ID

In [11]:
# filter companyID from Link

pattern = r"\d{8}"
df_admin['CompanyID'] = df_admin['Link'].apply(lambda x: re.findall(pattern,x)[0])
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301


#### 2.2.4 Clean column 'Company' 

In [12]:
# To find 22 missing value records in Column 'Company'
df_admin[df_admin['Company'].isnull()].head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: CanterburyCanterburyarea: ChristchurchChristchurch,"4d ago,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration,NaN,50622432
19,Administrator,https://www.seek.co.nz/job/50604829?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,NaN,location: ManawatuManawatuarea: Rest of ManawatuRest of Manawatu,"7d ago,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50604829
60,Administrator,https://www.seek.co.nz/job/50582301?type=standout#searchRequestToken=3aa68777-3165-4e73-98ef-bae1b5e67bc7,NaN,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"11d ago,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301
64,Administration Specialist - Construction,https://www.seek.co.nz/job/50638765?type=standout#searchRequestToken=3aa68777-3165-4e73-98ef-bae1b5e67bc7,NaN,location: AucklandAucklandarea: Rodney & North ShoreRodney & North Shore,"5h ago,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50638765


#### I found that 22 null in company because they are private advertisers
#### Next, I will fill "Private Advertiser" in them

In [13]:
# fill null value by 'Private Advertiser'
df_admin['Company'] = df_admin['Company'].fillna(value = 'Private Advertiser')
df_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Position        2708 non-null   object
 1   Link            2708 non-null   object
 2   Company         2708 non-null   object
 3   Location        2708 non-null   object
 4   Release_time    2708 non-null   object
 5   Classification  2708 non-null   object
 6   Worktype        454 non-null    object
 7   CompanyID       2708 non-null   object
dtypes: object(8)
memory usage: 169.4+ KB


#### 2.2.5 Deal with column 'Location'
#### inlcude wage data, extra word like"location:"

In [14]:
# check number of wage data inlcuded in Column'Location'
print(df_admin['Location'].str.contains(',').sum()) # 603

603


#### 603 Location data include wage

In [15]:
# remove salary inforamtion from Column location
df_admin['Salary'] = df_admin['Location'].apply(lambda x: x.split(',')[1] if "," in x else "")

In [16]:
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,


In [17]:
# delete wage data in Column 'Locaiton'
df_admin['Location'] = df_admin['Location'].apply(lambda x: x.split(',')[0] if "," in x else x)

In [18]:
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,location: Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,


In [19]:
 # remove extra word "location:"
df_admin['Location'] = df_admin['Location'].apply(lambda x: x[9:])

In [20]:
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,


In [21]:
# split location by word"area:" and put province into Column Area
df_admin['Area'] = df_admin['Location'].apply(lambda x: x.split("area:")[0] if ":" in x else x)

In [22]:
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of PlentyBay of Plenty


In [23]:
# split location by word"area:" and put city/region into Column City
df_admin['City'] = df_admin['Location'].apply(lambda x: x.split("area:")[1] if ":" in x else x)

In [24]:
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of PlentyBay of Plenty,TaurangaTauranga


In [25]:
# drop repeat word
def drop_repeat(s):
    trim = s.strip()
    index = (trim + trim).find(trim,1)
    if index == -1:
        return trim
    else:
        return trim[:index]
  

In [26]:
# delete repeat words
df_admin['Area'] = df_admin['Area'].apply(lambda x: drop_repeat(x))
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,TaurangaTauranga


In [27]:
# delete duplicat words
df_admin['City'] = df_admin['City'].apply(lambda x: drop_repeat(x))
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,"Featured,at,Private Advertiser",classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga


#### 2.2.6 Deal with column 'Release_time'
#### non useful word like 'at', 'Private Advertiser' need be deleted

In [28]:
# remove "Private advertiser or at" in Column Release_time
df_admin['Release_time'] = df_admin['Release_time'].apply(lambda x: x.split(',')[0])

In [29]:
df_admin.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,Featured,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,Featured,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists,NaN,50620889,,Bay of Plenty,Tauranga
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,AucklandAuckland,4d ago,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50622169,,Auckland,Auckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1h ago,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50639620,,Southland,Invercargill
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristchurch,4d ago,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration,NaN,50622432,,Canterbury,Christchurch


In [30]:
# remove "ago"
df_admin['Release_time'] = df_admin['Release_time'].apply(lambda x: "30d" if x.strip() == 'Featured' else x.split(" ")[0])

In [31]:
df_admin.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists,NaN,50620889,,Bay of Plenty,Tauranga
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,AucklandAuckland,4d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50622169,,Auckland,Auckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1h,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50639620,,Southland,Invercargill
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristchurch,4d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration,NaN,50622432,,Canterbury,Christchurch


In [32]:
# reocord less than 1 day as 1 day.
df_admin['Release_time'] = df_admin['Release_time'].apply(lambda x: "1d" if 'h' in x else x)

In [33]:
df_admin.head(30)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists,NaN,50620889,,Bay of Plenty,Tauranga
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,AucklandAuckland,4d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50622169,,Auckland,Auckland
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50639620,,Southland,Invercargill
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristchurch,4d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration,NaN,50622432,,Canterbury,Christchurch
5,Support Officer,https://www.seek.co.nz/job/50640393?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Ministry for Primary Industries,NorthlandNorthlandarea: WhangareiWhangarei,9m,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50640393,,Northland,Whangarei
6,Support Officer,https://www.seek.co.nz/job/50615674?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,"Ministry of Business, Innovation and Employment",WellingtonWellingtonarea: Wellington CentralWellington Central,5d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50615674,,Wellington,Wellington Central
7,Office Administrator,https://www.seek.co.nz/job/50640166?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Hepburn Electrical Ltd,Bay of PlentyBay of Plentyarea: RotoruaRotorua,7m,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50640166,,Bay of Plenty,Rotorua
8,Office Administrator,https://www.seek.co.nz/job/50639248?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Webster Holland Ltd,Bay of PlentyBay of Plentyarea: TaurangaTauranga,1d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50639248,,Bay of Plenty,Tauranga
9,Administration Officer,https://www.seek.co.nz/job/50629393?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,CanterburyCanterbury,3d,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50629393,,Canterbury,Canterbury


In [34]:
df_admin['Release_time'] = df_admin['Release_time'].apply(lambda x : "".join(re.findall('\d+d',x))[:-1])

In [35]:
df_admin.head(1)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga


#### 2.2.7 Deal with Column Classification

In [36]:
df_admin['Class'] = df_admin.loc[df_admin['Classification'].str.contains('SupportsubClassification: '),"Classification"]

In [37]:
df_admin.head(5)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City,Class
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists,NaN,50620889,,Bay of Plenty,Tauranga,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,AucklandAuckland,4,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50622169,,Auckland,Auckland,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50639620,,Southland,Invercargill,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,CanterburyCanterburyarea: ChristchurchChristchurch,4,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration,NaN,50622432,,Canterbury,Christchurch,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Client & Sales AdministrationClient & Sales Administration


In [38]:
df_admin['Class'].fillna("Unknown",inplace = True)

In [39]:
df_admin['Class'].isnull().sum()

0

In [40]:
df_admin['Class'] = df_admin['Class'].apply(lambda x: x if x == 'Unknown' else x.split("SupportsubClassification: ")[1])

In [41]:
df_admin.head(3)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City,Class
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga,Office ManagementOffice Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists,NaN,50620889,,Bay of Plenty,Tauranga,ReceptionistsReceptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,AucklandAuckland,4,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50622169,,Auckland,Auckland,OtherOther


In [42]:
df_admin['Class'] = df_admin['Class'].apply(lambda x: drop_repeat(x))
df_admin.head(4)

,Position,Link,Company,Location,Release_time,Classification,Worktype,CompanyID,Salary,Area,City,Class
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Office ManagementOffice Management,NaN,50582301,,Bay of Plenty,Tauranga,Office Management
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of PlentyBay of Plentyarea: TaurangaTauranga,30,classification: Administration & Office SupportAdministration & Office SupportsubClassification: ReceptionistsReceptionists,NaN,50620889,,Bay of Plenty,Tauranga,Receptionists
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,AucklandAuckland,4,classification: Administration & Office SupportAdministration & Office SupportsubClassification: OtherOther,NaN,50622169,,Auckland,Auckland,Other
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,SouthlandSouthlandarea: InvercargillInvercargill,1,classification: Administration & Office SupportAdministration & Office SupportsubClassification: Administrative AssistantsAdministrative Assistants,NaN,50639620,,Southland,Invercargill,Administrative Assistants


2.2.8 Deal with Column salary

In [43]:
df_admin['Salary'].unique()
print(df_admin['Salary'].value_counts())

                                                      2105
Competitive hourly rates $$                            280
Competitive hourly rate                                 71
Competitive hourly rate depending on experience         70
Attractive remuneration                                 70
$20 - $24.99 per hour                                   12
$50                                                      8
$55                                                      7
$40                                                      5
$45                                                      5
$20 - $29.99 per hour                                    3
Includes life and health insurance                       3
$35 - $40 p.h. + + 8% Holiday Pay                        2
Competitive                                              2
$60                                                      2
$24 - $28 p.h.  + 8% Holiday Pay                         2
$25 - $29.99 per hour                                   

In [44]:
df_admin[df_admin['Salary'].str.contains('\d',na = False)]['Salary']

12                                            $20 per hour
17                                   $20 - $24.99 per hour
23                                     $19 to $22 per hour
25                                   $20 - $24.99 per hour
36                                   $20 - $24.99 per hour
42                                                     $40
48                                            $25 per hour
58                             $60 000 - $70 000 per annum
59                                         $20.00 per hour
66                                   $20 - $24.99 per hour
75                                                     $55
77                                        $20.00 per hour.
78                                   $20 - $24.99 per hour
81                                            $21 per hour
82                                                     $55
89                                                     $50
97                                                     $

In [45]:
df_admin['Salary'] = df_admin.loc[df_admin['Salary'].str.contains('\d+'),'Salary']

In [46]:
df_admin['Salary'].unique()

array([nan, '$20 per hour', '$20 - $24.99 per hour',
       '$19 to $22 per hour', '$40', '$25 per hour',
       '$60 000 - $70 000 per annum', '$20.00 per hour', '$55',
       '$20.00 per hour.', '$21 per hour', '$50', '$45', '$48',
       '$20 - $29.99 per hour', '$24-$28', 'Up to $27ph',
       '8% holiday pay', 'NZD25 - NZD28 per hour', '$22 per hour',
       'NZD75000 per annum', '$75k - $85k p.a.',
       '$24 - $25 + 8% holiday pay!', '$20 - $25 per hour',
       '$30 - $38 p.h. + 8% Holiday Pay', '$45-$55',
       'Up to $23 p.h. + + 8% Holiday Pay', 'Up to $22.00 p.h.', 'NZD69',
       '$60k-$70k depending on experience!', '$22.00 - $23.00 phr',
       '$27 - $33 p.h. + 8% holiday pay',
       '$60K - $80K experience dependent',
       '$60 - $65k + Career Progression to QS / leadership',
       '$35 - $39.99 per hour',
       '$20 - $24.99 per hour dependent on experience.',
       'Up to $27 p.h. + 8% Holiday Pay', 'NZD19 - NZD23 per hour', '$60',
       '$21 - $25 p.h. + 8%

In [47]:
df_admin['Salary'] = df_admin['Salary'].str.replace('to','-',regex = True)

In [48]:
df_admin['Salary'].head(20)

0                       NaN
1                       NaN
2                       NaN
3                       NaN
4                       NaN
5                       NaN
6                       NaN
7                       NaN
8                       NaN
9                       NaN
10                      NaN
11                      NaN
12             $20 per hour
13                      NaN
14                      NaN
15                      NaN
16                      NaN
17    $20 - $24.99 per hour
18                      NaN
19                      NaN
Name: Salary, dtype: object

In [49]:
df_admin[['Low salary','High salary']] = df_admin['Salary'].str.split('-',n=1,expand = True).fillna('Unknown')

In [50]:
df_admin[['Low salary','High salary','Salary']]

,Low salary,High salary,Salary
0,Unknown,Unknown,NaN
1,Unknown,Unknown,NaN
2,Unknown,Unknown,NaN
3,Unknown,Unknown,NaN
4,Unknown,Unknown,NaN
5,Unknown,Unknown,NaN
6,Unknown,Unknown,NaN
7,Unknown,Unknown,NaN
8,Unknown,Unknown,NaN
9,Unknown,Unknown,NaN


In [51]:
df_admin['Low salary'] = df_admin['Low salary'].str.extract('(\d+\.\d+|\d+ \d+|\d+k|\d+)')
df_admin['High salary'] = df_admin['High salary'].str.extract('(\d+\.\d+|\d+ \d+|\d+k|\d+)')

In [52]:
df_admin[['Low salary','High salary','Salary']]

,Low salary,High salary,Salary
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [67]:
df_admin['Low salary'] = df_admin['Low salary'].str.replace('k','000',regex = True)
df_admin['High salary'] = df_admin['High salary'].str.replace('k','000',regex = True)

In [68]:
df_admin.loc[306:307,['Salary','Low salary','High salary']]

,Salary,Low salary,High salary
306,$75k - $85k p.a.,75000,85000
307,NaN,NaN,NaN


In [76]:
df_admin['Low salary'] = df_admin['Low salary'].str.replace(' ','',regex = True)
df_admin['High salary'] = df_admin['High salary'].str.replace(' ','',regex = True)

In [77]:
df_admin[['Low salary','High salary','Salary']]

,Low salary,High salary,Salary
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [78]:
df_admin['Low salary'] = df_admin['Low salary'].astype("float32")
df_admin['High salary'] = df_admin['High salary'].astype("float32")

In [79]:
df_admin['Low salary'] = df_admin.loc[df_admin['Salary'].str.contains('8%',na = False),'Low salary'] * 1.08

In [82]:
df_admin['High salary'] = df_admin.loc[df_admin['Salary'].str.contains('8%',na = False),'High salary'] * 1.08

In [83]:
df_admin.loc[384:403,['Low salary','High salary','Salary']]

,Low salary,High salary,Salary
384,32.400002,41.040001,$30 - $38 p.h. + 8% Holiday Pay
385,NaN,NaN,NaN
386,NaN,NaN,NaN
387,NaN,NaN,NaN
388,NaN,NaN,NaN
389,NaN,NaN,NaN
390,NaN,NaN,NaN
391,NaN,NaN,NaN
392,NaN,NaN,NaN
393,NaN,NaN,$45-$55


In [89]:
df_admin.to_excel('cleand_NZ_admin_jobs_data.xlsx')